# Social Distance Detection using scipy.spatial and OpenCV
# The sparks foundation
# Social Distancing detector
# Author- Hemant

# Importing libraries and ssd_mobilenet_v3 configs

In [1]:
import cv2
import matplotlib.pyplot as plt
from scipy.spatial import distance

In [2]:
frozen = 'frozen_inference_graph.pb'
config = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
features = []
model = cv2.dnn_DetectionModel(frozen, config)

In [3]:
label_file = 'labels.txt'
with open(label_file, 'rt') as f:
    features = f.read().rstrip('\n').split('\n')
    if features == 'person':
        desig = 'person'

In [4]:
print(features)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [5]:
model.setInputSize(320,320)
model.setInputScale(1.0/127.5)
model.setInputMean((127.5,127.5,127.5))
model.setInputSwapRB(True)
font_scale = 1
font_type = cv2.FONT_HERSHEY_PLAIN
rec = 'pedestrians walking.mp4'

In [6]:
ls = []
violate = set()
cam = cv2.VideoCapture(rec)
if not cam.isOpened():
    cam = cv2.VideoCapture(rec)
    if not cam.isOpened():
        print('Error in the file.........')
while(True):
    ret, frame = cam.read()
    frame = cv2.resize(frame, (720, 480), fx = 0, fy = 0, interpolation = cv2.INTER_CUBIC)
    classindex, confidence, bbox = model.detect(frame, confThreshold = 0.63)
    if len(classindex) != 0:
        for classind, conf, box in zip(classindex.flatten(), confidence.flatten(), bbox):
            if classind == 1:
                ls.append(box)
                cv2.rectangle(frame, box, (0, 255, 255), 2)
                cv2.putText(frame, features[classind - 1], (box[0] + 10, box[1] + 40), 
                            font_type, fontScale = font_scale, color = (255, 0, 0), thickness = 3)

    cv2.imshow('Frame', frame)
    if cv2.waitKey(2) & 0xff == ord('q'):
        break
cam.release()
cv2.destroyAllWindows()

In [7]:
centroid = []
min_distance = 50
for i in range(len(ls)):
    x, y, w, h = ls[i][0], ls[i][1], ls[i][2], ls[i][3]
    centroid.append([x, y])

dist = distance.cdist(centroid, centroid, metric = 'euclidean')
for i in range(0, dist.shape[0]):
    for j in range(i + 1, dist.shape[1]):
        if dist[i, j] < min_distance:
            violate.add(dist[i,j])

In [8]:
cam = cv2.VideoCapture(rec)
if not cam.isOpened():
    cam = cv2.VideoCapture(rec)
    if not cam.isOpened():
        print('Error in the file.........')
while(True):
    ret, frame = cam.read()
    frame = cv2.resize(frame, (720, 480), fx = 0, fy = 0, interpolation = cv2.INTER_CUBIC)
    classindex, confidence, bbox = model.detect(frame, confThreshold = 0.63)
    if len(classindex) > 0:
        for (i, (classind, conf, box)) in enumerate(zip(classindex.flatten(), confidence.flatten(), bbox)):
            if classind == 1:
                color = (0,255, 255)
                if i in violate:
                    color = (0, 0, 255)
                cv2.rectangle(frame, box, color, 3)
                cv2.putText(frame, features[classind - 1], (box[0] + 10, box[1] + 40), 
                                    font_type, fontScale = font_scale, color = (255, 0, 0), thickness = 3)

    text = f'Social Distancing Violations: {i}'
    cv2.putText(frame, text, (10, frame.shape[0] - 25), font_type, fontScale = font_scale,
                color = (0, 50, 255), thickness = 2)

    cv2.imshow('Frame', frame)
    if cv2.waitKey(2) & 0xff == ord('q'):
        break
cam.release()
cv2.destroyAllWindows()